In [11]:
# quickcommerce_simulation.py
import os
import pandas as pd
import numpy as np
import random
import calendar
import logging
from faker import Faker
from datetime import datetime, timedelta

## Main Code

In [12]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date, time
import random
import os
import calendar
import logging
from faker import Faker

# -----------------------
# Logging Setup
# -----------------------
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s:%(message)s')

# -----------------------
# Global Setup & Constants
# -----------------------
np.random.seed(150)
random.seed(150)
fake = Faker('en_IN')

DEFAULT_START_DATE = '2022-01-01'

# Configurable feedback data path
FEEDBACK_DATA_PATH = os.environ.get(
    'FEEDBACK_DATA_PATH',
    r"C:\DATA SCIENCE ROADMAP\PROJECTS\Apna Quick Commerce Project (PowerBI+MySQL)\New data\realistic_feedback.csv"
)

# File paths
ORDERS_CSV = 'Apna_orders.csv'
ORDER_ITEMS_CSV = 'Apna_order_items.csv'
DELIVERY_PERFORMANCE_CSV = 'Apna_delivery_performance.csv'
CUSTOMER_FEEDBACK_CSV = 'Apna_customer_feedback.csv'
STATIC_CUSTOMERS_CSV = 'Apna_customers.csv'
STATIC_PRODUCTS_CSV = 'Apna_products.csv'
MARKETING_PERFORMANCE_CSV = 'Apna_marketing_performance.csv'
INVENTORY_CSV = 'Apna_inventory.csv'

# Load realistic feedback data
try:
    realistic_feedback_df = pd.read_csv(FEEDBACK_DATA_PATH)
    # Expecting columns: id, rating, feedback_text, feedback_category, sentiment, order_status
except Exception as e:
    logging.error(f"Error loading feedback data: {e}")
    realistic_feedback_df = pd.DataFrame(
        columns=['id','rating', 'feedback_text', 'feedback_category', 'sentiment', 'order_status']
    )

# -----------------------
# Define Base & Cancel Categories
# -----------------------
base_categories = [
    'Product Quality',
    'Customer Service',
    'Fast Delivery',
    'Slow Delivery',
    'App Experience',
    'Price Concern',
    'Packaging Issue',
    'Payment Trouble',
    'Delivery Concern'
]

cancel_reasons = [
    'Product Unavailable',
    'Customer Cancellation',
    'Payment Issue',
    'Changed Mind',
    'Found Cheaper Alternative',
    'Price Issue',
    'Stock Issue'
]

# -----------------------
# Expanded City, Delivery Partner & Pincode Settings
# -----------------------
CITY_LIST = [
    "Agra","Ahmedabad","Allahabad","Amritsar","Asansol","Bengaluru","Bhopal","Bhubaneswar","Chandigarh","Chennai","Dehradun","Delhi","Goa","Guwahati","Howrah","Hyderabad","Indore","Jaipur","Jalandhar","Kanpur","Kolkata","Kota","Lucknow","Ludhiana","Mumbai","Mysuru","Nagpur","Nasik","New Delhi","Pune","Rajkot","Ranchi","Surat","Tiruchirapalli","Vadodara","Varanasi","Vijayawada" 
]

CITY_WEIGHTS = [5, 15, 5, 5, 5, 20, 10, 8, 8, 20, 5, 20, 5, 8, 5, 20, 8, 10, 5, 10, 15, 5, 10, 10, 20, 5, 8, 8, 20, 15, 5, 5, 15, 5, 8, 5, 5]


city_delivery_partners = {"Agra": list(range(200, 250)), "Ahmedabad": list(range(250, 300)), "Allahabad": list(range(300, 350)), "Amritsar": list(range(350, 400)), "Asansol": list(range(400, 450)), "Bengaluru": list(range(450, 500)), "Bhopal": list(range(500, 550)), "Bhubaneswar": list(range(550, 600)), "Chandigarh": list(range(600, 650)), "Chennai": list(range(650, 700)), "Dehradun": list(range(700, 750)), "Delhi": list(range(750, 800)), "Goa": list(range(800, 850)), "Guwahati": list(range(850, 900)), "Howrah": list(range(900, 950)), "Hyderabad": list(range(950, 1000)), "Indore": list(range(1000, 1050)), "Jaipur": list(range(1050, 1100)), "Jalandhar": list(range(1100, 1150)), "Kanpur": list(range(1150, 1200)), "Kolkata": list(range(1200, 1250)), "Kota": list(range(1250, 1300)), "Lucknow": list(range(1300, 1350)), "Ludhiana": list(range(1350, 1400)), "Mumbai": list(range(1400, 1450)), "Mysuru": list(range(1450, 1500)), "Nagpur": list(range(1500, 1550)), "Nasik": list(range(1550, 1600)), "New Delhi": list(range(1600, 1650)), "Pune": list(range(1650, 1700)), "Rajkot": list(range(1700, 1750)), "Ranchi": list(range(1750, 1800)), "Surat": list(range(1800, 1850)), "Tiruchirapalli": list(range(1850, 1900)), "Vadodara": list(range(1900, 1950)), "Varanasi": list(range(1950, 2000)), "Vijayawada": list(range(2000, 2050))}

store_ids = {"Agra": list(range(20000,20050)), "Ahmedabad": list(range(20050,20100)), "Allahabad": list(range(20100,20150)), "Amritsar": list(range(20150,20200)), "Asansol": list(range(20200,20250)), "Bengaluru": list(range(20250,20300)), "Bhopal": list(range(20300,20350)), "Bhubaneswar": list(range(20350,20400)), "Chandigarh": list(range(20400,20450)), "Chennai": list(range(20450,20500)), "Dehradun": list(range(20500,20550)), "Delhi": list(range(20550,20600)), "Goa": list(range(20600,20650)), "Guwahati": list(range(20650,20700)), "Howrah": list(range(20700,20750)), "Hyderabad": list(range(20750,20800)), "Indore": list(range(20800,20850)), "Jaipur": list(range(20850,20900)), "Jalandhar": list(range(20900,20950)), "Kanpur": list(range(20950,21000)), "Kolkata": list(range(21000,21050)), "Kota": list(range(21050,21100)), "Lucknow": list(range(21100,21150)), "Ludhiana": list(range(21150,21200)), "Mumbai": list(range(21200,21250)), "Mysuru": list(range(21250,21300)), "Nagpur": list(range(21300,21350)), "Nasik": list(range(21350,21400)), "New Delhi": list(range(21400,21450)), "Pune": list(range(21450,21500)), "Rajkot": list(range(21500,21550)), "Ranchi": list(range(21550,21600)), "Surat": list(range(21600,21650)), "Tiruchirapalli": list(range(21650,21700)), "Vadodara": list(range(21700,21750)), "Varanasi": list(range(21750,21800)), "Vijayawada": list(range(21800,21850))}

city_pincode_ranges = {"Agra": (282001, 283202),"Ahmedabad": (380001, 382480),"Allahabad": (211001, 274308),"Amritsar":(143001, 143606),"Asansol": (713201, 713386),"Bengaluru": (560001, 560500),"Bhopal": (462001, 464993),"Bhubaneswar": (751001, 754012),"Chandigarh": (140001, 160104),"Chennai": (600001, 600119),"Dehradun": (247656, 249411),"Delhi": (110006, 110099),"Goa": (403001, 403806),"Guwahati": (781001, 782403),"Howrah": (711101, 711414),"Hyderabad": (500001, 501512),"Indore": (452001, 452018),"Jaipur": (302001, 302043),"Jalandhar": (144001, 144518),"Kanpur": (208001, 209402),"Kolkata": (700001, 700163),"Kota": (324001, 328216),"Lucknow": (226001, 226501),"Ludhiana": (141001, 141017),"Mumbai": (400001, 400104),"Mysuru": (570001, 571617),"Nagpur": (440001, 440037),"Nasik": (422001, 422606),"New Delhi": (110001, 110097),"Pune": (411001, 411068),"Rajkot": (360001, 363670),"Ranchi": (834001, 835325),"Surat": (394101, 395023),"Tiruchirapalli": (608901, 639115),"Vadodara": (390001, 392310),"Varanasi": (221001, 232120),"Vijayawada": (509132, 521457)}

# -----------------------
# Product Category Popularity & Customer Segmentation
# -----------------------
SEGMENTATION_WEIGHT = {
    "Premium": 1.0,
    "Regular": 1.5,
    "New": 0.5,
    "Inactive": 0.2
}

# -----------------------
# Global Inventory Dictionary
# -----------------------
inventory_dict = {}
products_dict = {}

# -----------------------
# Helper Function to Determine Season
# -----------------------
def get_season(current_date):
    m = current_date.month
    if m in (12, 1, 2): return 'winter'
    if m in (3, 4, 5): return 'spring'
    if m in (6, 7, 8): return 'summer'
    return 'autumn'

# -----------------------
# Customer Segmentation
# -----------------------
def update_segmentation(customer, current_date):
    reg = customer['registration_date']
    last = customer.get('last_order_date', reg)
    total = customer.get('total_orders', 0)
    days_since_reg = (current_date - reg).days
    days_since_last = (current_date - last).days if last else None
    if days_since_reg <= 60:
        seg = "New"
    elif days_since_last is not None and days_since_last > 120:
        seg = "Inactive"
    elif total >= 60:
        seg = "Premium"
    else:
        seg = "Regular"
    customer['segmentation'] = seg
    return customer

# -----------------------
# Time & Product Selection
# -----------------------
def weighted_time_in_day(target_date):
    slots = [((0,0),(7,0),5),((7,0),(10,0),10),((10,0),(12,0),15),
             ((12,0),(14,0),10),((14,0),(18,0),20),((18,0),(21,0),30),((21,0),(23,59),10)]
    chosen = random.choices(slots, weights=[s[2] for s in slots], k=1)[0]
    start = datetime.combine(target_date, time(*chosen[0]))
    end   = datetime.combine(target_date, time(*chosen[1]))
    sec = max(1, int((end-start).total_seconds()))
    return start + timedelta(seconds=random.randint(0,sec))

def select_product_for_order(products):
    avail = [p for p in products if inventory_dict.get(p['product_id'],{}).get('current_stock',0)>0]
    if not avail: return None
    weights = {
        'Grocery & Staples':25,'Dairy & Breakfast':18,'Fruits & Vegetables':15,
        'Snacks & Munchies':10,'Cold Drinks & Juices':6,'Instant & Frozen Food':9,
        'Household Care':5,'Personal Care':3,'Baby Care':2,'Pet Care':1,'Pharmacy':6
    }
    return random.choices(avail, weights=[weights.get(p['category'],1) for p in avail], k=1)[0]

# -----------------------
# Growth & Orders Target
# -----------------------
growth_factors = {}
def get_growth_factor(year, day_type):
    if year not in growth_factors:
        if year == 2022:
            growth_factors[2022] = (1.0,1.0)
        else:
            prev = year-1
            _ = get_growth_factor(prev,'weekday')
            _ = get_growth_factor(prev,'weekend')
            w = min(growth_factors[prev][0]*random.uniform(1.23,1.28),10)
            e = min(growth_factors[prev][1]*random.uniform(1.25,1.33),10)
            growth_factors[year]=(w,e)
    return growth_factors[year][0] if day_type=='weekday' else growth_factors[year][1]

def get_orders_target_for_date(target_date):
    dt = target_date.weekday()
    dt_type = 'weekday' if dt<5 else 'weekend'
    base = random.randint(160,200) if dt_type=='weekday' else random.randint(185,245)
    fac = get_growth_factor(target_date.year,dt_type)
    raw = base*fac
    days = calendar.monthrange(target_date.year,target_date.month)[1]
    mf = 1.0-0.3*((target_date.day-1)/(days-1))
    return max(1,int(round(raw*mf)))

# -----------------------
# Dynamic Pricing
# -----------------------
def compute_dynamic_price(product, available_stock, current_date, days_since_replenishment=None):
    mrp0 = product['mrp']
    max_s = product['max_stock_level'] or 1
    ratio = available_stock / max_s
    if ratio < 0.2:
        price = mrp0 * random.uniform(1.10, 1.25)
    elif ratio < 0.4:
        price = mrp0 * random.uniform(1.05, 1.10)
    elif ratio > 0.8:
        price = mrp0 * random.uniform(0.75, 0.90)
    elif ratio > 0.6:
        price = mrp0 * random.uniform(0.90, 0.95)
    else:
        price = mrp0 * random.uniform(0.97, 1.03)
    if days_since_replenishment is not None and days_since_replenishment >= 0.8 * product['shelf_life_days']:
        price *= (1 - random.uniform(0.12, 0.20))
    return round(price)


# -----------------------
# Inventory Functions
# -----------------------
def initialize_inventory(products_df, start_date):
    inv_records = []
    d0 = datetime.strptime(DEFAULT_START_DATE, '%Y-%m-%d').date()
    for _, row in products_df.iterrows():
        pid = row['product_id']
        init_stock = 0 if start_date == DEFAULT_START_DATE else random.randint(row['min_stock_level'], row['max_stock_level'])
        base_cost = row['base_purchasing_cost']
        purchasing_cost_today = base_cost
        price_today = compute_dynamic_price(row, init_stock, d0, days_since_replenishment=0)
        inventory_dict[pid] = {
            'current_stock': init_stock,
            'min_stock_level': row['min_stock_level'],
            'max_stock_level': row['max_stock_level'],
            'last_replenishment_date': d0,
            'new_stock': init_stock,
            'damaged_stock': 0,
            'latest_price': price_today,
            'purchasing_cost': purchasing_cost_today,
            'prev_day_end_stock': init_stock,
            'sold_stock': 0
        }
        inv_records.append({
            'product_id': pid,
            'date': d0,
            'mrp': row['mrp'],
            'price': price_today,
            'purchasing_cost': purchasing_cost_today,
            'Selling Cost': 0.0,
            'profit': 0.0,
            'new_stock': init_stock,
            'start_available_stock': init_stock,
            'sold_stock': 0,
            'end_available_stock': init_stock,
            'damaged_stock': 0
        })
    return pd.DataFrame(inv_records)

def update_inventory_record_for_day(current_date, products_dict, inventory_dict):
    records = []
    for pid, inv in inventory_dict.items():
        prod = products_dict[pid]
        days_since_repl = (current_date - inv['last_replenishment_date']).days
        # Spoilage
        damaged = 0
        if days_since_repl >= prod['shelf_life_days']:
            damaged = inv['current_stock']
            inv['current_stock'] = 0
        inv['damaged_stock'] = damaged
        # Restock
        new_stock_qty = 0
        if inv['current_stock'] < prod['min_stock_level']:
            target = prod['max_stock_level']
            replen_qty = (target - inv['current_stock']) + random.randint(-3, 3)
            replen_qty = max(replen_qty, 0)
            if replen_qty > 0:
                replenishment_quantity = replen_qty
                new_stock_qty = replenishment_quantity
                inv['current_stock'] += replen_qty
                inv['last_replenishment_date'] = current_date
        inv['new_stock'] = new_stock_qty
        # Start available
        if 'prev_day_end_stock' in inv:
            start_available = inv['prev_day_end_stock'] + new_stock_qty
        else:
            start_available = inv['current_stock'] + inv['sold_stock'] + damaged - new_stock_qty
        inv['start_available_stock'] = start_available
        # Sold
        sold = inv['sold_stock']
        # End available
        end_stock = inv['current_stock']
        inv['end_available_stock'] = end_stock
        # Purchasing cost (season + wiggle)
        base_cost = prod['base_purchasing_cost']
        season = get_season(current_date)
        if season == 'summer':
            season_mult = random.uniform(1.03, 1.10)
        elif season == 'winter':
            season_mult = random.uniform(0.90, 0.98)
        else:
            season_mult = random.uniform(0.97, 1.03)
        wiggle_pct = random.uniform(-0.03, 0.03)
        purchase_cost = round(base_cost * season_mult * (1 + wiggle_pct), 2)
        inv['purchasing_cost'] = purchase_cost
        # Raw dynamic price
        raw_price = compute_dynamic_price(prod, start_available, current_date, days_since_repl)
        # Bump MRP if raw_price > static MRP
        today_mrp = prod['mrp']
        if raw_price > today_mrp:
            today_mrp = raw_price
        # Selling cost per unit (5-15% of purchase)
        selling_cost = max(purchase_cost * random.uniform(0.06, 0.10),3)
        inv['selling_cost_per_unit'] = selling_cost
        # Final price enforce floor and cap
        floor_price = purchase_cost + selling_cost
        final_price = max(raw_price, floor_price)
        final_price = min(final_price, today_mrp)
        inv['price'] = round(final_price, 2)
        # Profit per unit
        unit_profit = round(final_price - (purchase_cost + selling_cost), 2)
        # Build record in desired column order
        row = {
            'product_id': pid,
            'date': current_date,
            'mrp': today_mrp,
            'price': inv['price'],
            'purchasing_cost': purchase_cost,
            'Selling Cost': selling_cost,
            'profit': unit_profit,
            'new_stock': new_stock_qty,
            'start_available_stock': start_available,
            'sold_stock': sold,
            'end_available_stock': end_stock,  # <-- changed from 'end_available_stock'
            'damaged_stock': damaged
        }
        records.append(row)
        # Store end-of-day
        inv['prev_day_end_stock'] = end_stock
        inv['sold_stock'] = 0
    return records


# -----------------------
# Order Generation Helpers
# -----------------------

def get_new_date_range(path, default):
    if os.path.exists(path):
        try:
            df=pd.read_csv(path,parse_dates=['order_date'])
        except:
            df=pd.DataFrame()
        if not df.empty:
            last=df['order_date'].max().date()
            start=last+timedelta(days=1)
        else:
            start=datetime.strptime(default,'%Y-%m-%d').date()
    else:
        start=datetime.strptime(default,'%Y-%m-%d').date()
    return start, date.today()


def generate_orders_for_day(target_date, eligible_customers, products, customers_dict,
                            oid_start, fid_start, orders_target):
    orders = []
    items_list = []
    perf = []
    feedback = []

    order_id = oid_start
    fid = fid_start
    seg_w = [SEGMENTATION_WEIGHT.get(c.get('segmentation','Regular'),1) for c in eligible_customers]
    gen = 0

    while gen < orders_target:
        cust = random.choices(eligible_customers, weights=seg_w, k=1)[0]
        cid = cust['customer_id']
        dp = city_delivery_partners.get(cust['city'])
        od = weighted_time_in_day(target_date)
        if od.date() < cust['registration_date']:
            od = datetime.combine(cust['registration_date'], od.time())
        prom = od + timedelta(minutes=random.randint(8,10))
        delay = random.randint(5,10) if cust['city'] in ["Mumbai","Delhi"] else 0

        # Build items
        num = random.randint(1,3)
        items = []
        total = 0.0
        canceled = False
        stock_canceled = False
        for _ in range(num):
            prod = select_product_for_order(products)
            if prod is None:
                canceled = True
                stock_canceled = True
                break
            pid = prod['product_id']
            cs = inventory_dict[pid]['current_stock']
            if cs < 1:
                canceled = True
                stock_canceled = True
                break
            qty = random.randint(1, min(3, cs))
            up = inventory_dict[pid]['latest_price']
            items.append({
                'order_id': order_id,
                'product_id': pid,
                'quantity': qty,
                'unit_price': up,
                'total_price': round(up * qty, 2)
            })
            total += up * qty
            inventory_dict[pid]['current_stock'] -= qty
            inventory_dict[pid].setdefault('sold_stock', 0)
            inventory_dict[pid]['sold_stock'] += qty

        # Random cancellation
        if random.random() < 0.04:
            canceled = True

        # Roll back and reset on cancellation
        if canceled:
            for it in items:
                pid = it['product_id']
                inventory_dict[pid]['current_stock'] += it['quantity']
                inventory_dict[pid]['sold_stock'] -= it['quantity']
            total = 0.0  # <<< Reset total when canceled

        # Determine final status and delivery times
        if canceled:
            status = 'Cancelled'
            dstatus = 'Cancelled'
            # Distinguish stock vs. user/random cancellations:
            if stock_canceled:
                reason = "Product Unavailable"
            else:
                reason = random.choices(
                    ['Customer Cancellation','Payment Issue','Changed Mind','Found Cheaper Alternative','Price Issue','Stock Issue'],weights=[17,18,13,15,22,9]
                )[0]
            ad = None
        else:
            status = 'Completed'
            r = random.random()
            if r < 0.87:
                ad = od + timedelta(minutes=random.randint(6,10) + delay)
                dstatus = 'On Time'
                reason = None
            elif r < 0.93:
                ad = od + timedelta(minutes=random.randint(11,22) + delay)
                dstatus = 'Slightly Delayed'
                reason = random.choices(
                    ["Traffic", "Stock Issue", "Weather", "Technical Issue"],
                    weights=[33,37,25,15]
                )[0]
            else:
                ad = od + timedelta(minutes=random.randint(22,40) + delay)
                dstatus = 'Significantly Delayed'
                reason = random.choices(
                    ["Traffic", "Stock Issue", "Weather", "Technical Issue"],
                    weights=[39,26,22,15]
                )[0]
        sp = store_ids.get(cust['city']) 
        order_rec = {
            'order_id': order_id,
            'customer_id': cid,
            'order_date': od,
            'promised_delivery_time': prom,
            'actual_delivery_time': ad,
            'delivery_status': dstatus,
            'order_total': round(total, 2),
            'payment_method': random.choices(
                ['UPI','Card','COD','Wallet'], weights=[70,5,10,15]
            )[0],
            'delivery_partner_id': random.choice(dp),
            'store_id': random.choice(sp),
            'delay_reason': reason,
            'order_status': status
        }

        # 4) Always log the order header
        orders.append(order_rec)

        # 5) Always log all attempted items
        items_list.extend(items)

        # 6) Log performance (ETA/distance) even for cancelled
        perf.append({
            'order_id': order_id,
            'delivery_partner_id': random.choice(dp),
            'promised_time': prom,
            'actual_time': ad,
            'delivery_time_minutes': round((ad - od).total_seconds() / 60, 2) if ad else None,
            'distance_km': round(random.uniform(0.5,5), 2),
            'delivery_status': dstatus,
            'reasons_if_delayed': reason
        })

        # -------------------------------------------------------
        # 7) **FEEDBACK**: pull from realistic_feedback_df
        # -------------------------------------------------------
        if status == 'Cancelled':
            fr = realistic_feedback_df[
                (realistic_feedback_df['order_status'] == 'Cancelled') &
                (realistic_feedback_df['feedback_category'].isin(cancel_reasons))
            ]
        else:
            if dstatus == 'On Time':
                fr = realistic_feedback_df[
                    (realistic_feedback_df['order_status'] == 'Completed') &
                    (realistic_feedback_df['sentiment'] == 'Positive')
                ]
            elif dstatus == 'Slightly Delayed':
                fr = realistic_feedback_df[
                    (realistic_feedback_df['order_status'] == 'Completed') &
                    (realistic_feedback_df['feedback_category'] == 'Delivery Concern')
                ]
            else:  # Fully late or other negative experiences
                fr = realistic_feedback_df[
                    (realistic_feedback_df['order_status'] == 'Completed') &
                    (realistic_feedback_df['feedback_category'] == 'Slow Delivery')
                ]

        if not fr.empty:
            row = fr.sample().iloc[0]
            fb = {
                'feedback_id': fid,
                'order_id': order_id,
                'customer_id': cid,
                'rating': row['rating'],
                'feedback_text': row['feedback_text'],
                'feedback_category': row['feedback_category'],
                'sentiment': row['sentiment'],
                'feedback_date': (ad if ad else od) + timedelta(minutes=random.randint(10,60)),
                'order_status': status
            }
        else:
            if status == 'Cancelled':
                fb = {
                    'feedback_id': fid,
                    'order_id': order_id,
                    'customer_id': cid,
                    'rating': 1,
                    'feedback_text': 'Order cancelled',
                    'feedback_category': 'Cancellation',
                    'sentiment': 'Negative',
                    'feedback_date': od + timedelta(minutes=random.randint(10,60)),
                    'order_status': status
                }
            else:
                fallback_sent = 'Positive' if dstatus == 'On Time' else 'Neutral'
                fb = {
                    'feedback_id': fid,
                    'order_id': order_id,
                    'customer_id': cid,
                    'rating': 5 if dstatus == 'On Time' else 3,
                    'feedback_text': 'All good' if dstatus == 'On Time' else 'Delivery was late',
                    'feedback_category': 'Fast Delivery' if dstatus == 'On Time' else 'Slow Delivery',
                    'sentiment': fallback_sent,
                    'feedback_date': ad + timedelta(minutes=random.randint(10,60)),
                    'order_status': status
                }

        feedback.append(fb)
        fid += 1
        # -------------------------------------------------------

        # 8) Only update customer stats for completed orders
        if not canceled:
            old_tot = cust.get('total_orders', 0)
            old_avg = cust.get('avg_order_value', 0)
            new_tot = old_tot + 1
            new_avg = round(((old_avg * old_tot) + total) / new_tot, 2) if new_tot else total
            cust['total_orders']    = new_tot
            cust['avg_order_value'] = new_avg
            cust['last_order_date'] = od.date()
            customers_dict[cid]     = update_segmentation(cust, target_date)

        # 9) Advance IDs and counters
        order_id += 1
        if not canceled:
            gen += 1

    return orders, items_list, perf, feedback, order_id, fid


def generate_first_order_for_customer(customer, products, customers_dict, order_id, fid, target_date):
    cid = customer['customer_id']
    dp = city_delivery_partners.get(customer['city'])
    sp = store_ids.get(customer['city'])
    od = weighted_time_in_day(target_date)
    if od.date() != customer['registration_date']:
        od = datetime.combine(customer['registration_date'], od.time())
    prom = od + timedelta(minutes=random.randint(8,10))
    delay = random.randint(5,10) if customer['city'] in ["Mumbai","Delhi"] else 0

    num = random.randint(1,3)
    items = []
    total = 0.0
    canceled = False
    for _ in range(num):
        prod = select_product_for_order(products)
        if prod is None:
            canceled = True
            break
        pid = prod['product_id']
        cs = inventory_dict[pid]['current_stock']
        if cs < 1:
            canceled = True
            break
        qty = random.randint(1, min(3, cs))
        up = inventory_dict[pid]['latest_price']
        items.append({'order_id':order_id,'product_id':pid,'quantity':qty,'unit_price':up,'total_price':round(up*qty,2)})
        total += up * qty
        inventory_dict[pid]['current_stock'] -= qty
        inventory_dict[pid].setdefault('sold_stock',0)
        inventory_dict[pid]['sold_stock'] += qty

    if random.random() < 0.04:
        canceled = True

    if canceled:
        for it in items:
            pid = it['product_id']
            inventory_dict[pid]['current_stock'] += it['quantity']
            inventory_dict[pid]['sold_stock'] -= it['quantity']
        total = 0.0  # reset total on cancellation

    if canceled:
        status = 'Cancelled'
        dstatus = 'Cancelled'
        reason = "Product Unavailable" if canceled else random.choice(
            ["Customer Cancellation","Payment Issue","Changed Mind","Found Cheaper Alternative"]
        )
        ad = None
    else:
        status = 'Completed'
        r = random.random()
        if r < 0.87:
            ad = od + timedelta(minutes=random.randint(6,10)+delay)
            dstatus = 'On Time'
            reason = None
        elif r < 0.93:
            ad = od + timedelta(minutes=random.randint(11,22)+delay)
            dstatus = 'Slightly Delayed'
            reason = random.choices(
                ["Traffic","Stock Issue","Weather","Technical Issue"],
                weights=[30,30,25,15]
            )[0]
        else:
            ad = od + timedelta(minutes=random.randint(22,40)+delay)
            dstatus = 'Significantly Delayed'
            reason = random.choices(
                ["Traffic","Stock Issue","Weather","Technical Issue"],
                weights=[30,30,25,15]
            )[0]
   
    order_rec = {
        'order_id': order_id,
        'customer_id': cid,
        'order_date': od,
        'promised_delivery_time': prom,
        'actual_delivery_time': ad,
        'delivery_status': dstatus,
        'order_total': round(total,2),
        'payment_method': random.choices(['UPI','Card','COD','Wallet'],[70,5,10,15])[0],
        'delivery_partner_id': random.choice(dp),
        'store_id': random.choice(sp),
        'delay_reason': reason,
        'order_status': status
    }
    perf = {
        'order_id': order_id,
        'delivery_partner_id': random.choice(dp),
        'promised_time': prom,
        'actual_time': ad,
        'delivery_time_minutes': round((ad-od).total_seconds()/60,2) if ad else None,
        'distance_km': round(random.uniform(0.5,5),2),
        'delivery_status': dstatus,
        'reasons_if_delayed': reason
    }

    # -------------------------------------------------------
    # Feedback for first‐order: same logic as generate_orders_for_day
    # -------------------------------------------------------
    if status == 'Cancelled':
        fr = realistic_feedback_df[
            (realistic_feedback_df['order_status'] == 'Cancelled') &
            (realistic_feedback_df['feedback_category'].isin(cancel_reasons))
        ]
    else:
        if dstatus == 'On Time':
            fr = realistic_feedback_df[
                (realistic_feedback_df['order_status'] == 'Completed') &
                (realistic_feedback_df['sentiment'] == 'Positive')
            ]
        else:
            fr = realistic_feedback_df[
                (realistic_feedback_df['order_status'] == 'Completed') &
                (realistic_feedback_df['feedback_category'] == 'Slow Delivery')
            ]

    if not fr.empty:
        row = fr.sample().iloc[0]
        fb = {
            'feedback_id': fid,
            'order_id': order_id,
            'customer_id': cid,
            'rating': row['rating'],
            'feedback_text': row['feedback_text'],
            'feedback_category': row['feedback_category'],
            'sentiment': row['sentiment'],
            'feedback_date': (ad if ad else od) + timedelta(minutes=random.randint(10,60)),
            'order_status': status
        }
    else:
        if status == 'Cancelled':
            fb = {
                'feedback_id': fid,
                'order_id': order_id,
                'customer_id': cid,
                'rating': 1,
                'feedback_text': 'Order cancelled',
                'feedback_category': 'Cancellation',
                'sentiment': 'Negative',
                'feedback_date': od + timedelta(minutes=random.randint(10,60)),
                'order_status': status
            }
        else:
            fallback_sent = 'Positive' if dstatus == 'On Time' else 'Neutral'
            fb = {
                'feedback_id': fid,
                'order_id': order_id,
                'customer_id': cid,
                'rating': 5 if dstatus == 'On Time' else 3,
                'feedback_text': 'All good' if dstatus == 'On Time' else 'Delivery was late',
                'feedback_category': 'Fast Delivery' if dstatus == 'On Time' else 'Slow Delivery',
                'sentiment': fallback_sent,
                'feedback_date': ad + timedelta(minutes=random.randint(10,60)),
                'order_status': status
            }

    fid += 1
    # -------------------------------------------------------

    # Only update stats for completed
    if not canceled:
        old_tot = customer.get('total_orders', 0)
        old_avg = customer.get('avg_order_value', 0)
        new_tot = old_tot + 1
        new_avg = round(((old_avg * old_tot) + total) / new_tot, 2) if new_tot else total
        customer['total_orders']    = new_tot
        customer['avg_order_value'] = new_avg
        customer['last_order_date'] = od.date()
        customers_dict[cid] = update_segmentation(customer, target_date)

    return order_rec, items, perf, fb, order_id+1, fid+1

# -----------------------
# New Customer Simulation & Daily Loop
# -----------------------
def simulate_new_customers_for_day(current_date, num_new_customers, next_cid, emails, phones):
    new_cust=[]
    for _ in range(num_new_customers):
        raw = fake.address().replace("\n", ", ")
        # Split into components
        parts = [p.strip() for p in raw.split(",") if p.strip()]
        # street is first part, area is second if exists, else fallback
        street = fake.street_name()
        area = parts[1] if len(parts) > 1 else fake.street_name()
        house_num = fake.building_number()
        city = random.choices(CITY_LIST, weights=CITY_WEIGHTS)[0]
        pincode_low, pincode_high = city_pincode_ranges[city]
        pin = random.randint(pincode_low, pincode_high)
        address = f"{house_num} {street} , {area}"
        name = fake.name()

        # Email based on name
        local = name.lower().replace(" ", ".")
        domain = random.choice(["gmail.com", "yahoo.com", "outlook.com"])
        email = f"{local}{random.randint(1,999)}@{domain}"

        # Indian mobile number +91 and 10 digits starting 6–9
        phone = f"+91{random.randint(6000000000, 9999999999)}"
        
        cust={'customer_id':next_cid,'customer_name':name,'email':email,'phone':phone,'address':address,'area':city,'city':city,'pincode':pin,'registration_date':current_date,'total_orders':0,'avg_order_value':0,'last_order_date':current_date,'segmentation':'New'}
        new_cust.append(cust); next_cid+=1
    return new_cust, next_cid


def generate_new_daily_orders(new_start_date, new_end_date):
    # --- LOAD/INIT CUSTOMERS & PRODUCTS ---
    try:
        cust_df = pd.read_csv(STATIC_CUSTOMERS_CSV, parse_dates=['registration_date', 'last_order_date'])
    except Exception as e:
        logging.error(f"Error reading {STATIC_CUSTOMERS_CSV}: {e}")
        return {}

    cust_df['registration_date'] = cust_df['registration_date'].dt.date
    cust_df['last_order_date']   = cust_df['last_order_date'].dt.date
    customers_dict_local = {
        row['customer_id']: row.to_dict()
        for _, row in cust_df.iterrows()
    }
    global customers_dict
    customers_dict = customers_dict_local

    next_customer_id = max(customers_dict.keys()) + 1
    existing_emails = {c['email'] for c in customers_dict.values()}
    existing_phones = {c['phone'] for c in customers_dict.values()}

    if os.path.exists(STATIC_PRODUCTS_CSV) and os.path.getsize(STATIC_PRODUCTS_CSV) > 0:
        prod_df = pd.read_csv(STATIC_PRODUCTS_CSV)
    else:
        prod_df = generate_static_products()
        prod_df.to_csv(STATIC_PRODUCTS_CSV, index=False)
    products = prod_df.to_dict('records')
    global products_dict
    products_dict = {row['product_id']: row for row in products}

    if os.path.exists(ORDERS_CSV) and os.path.getsize(ORDERS_CSV) > 0:
        try:
            old_o = pd.read_csv(ORDERS_CSV)
            order_id_counter = int(old_o['order_id'].max()) + 1
        except:
            order_id_counter = 20000
    else:
        order_id_counter = 20000

    if os.path.exists(CUSTOMER_FEEDBACK_CSV) and os.path.getsize(CUSTOMER_FEEDBACK_CSV) > 0:
        try:
            old_f = pd.read_csv(CUSTOMER_FEEDBACK_CSV)
            feedback_id_counter = int(old_f['feedback_id'].max()) + 1
        except:
            feedback_id_counter = 1
    else:
        feedback_id_counter = 1

    all_orders = []
    all_items = []
    all_perf = []
    all_feedback = []
    inventory_records_list = []

    # --- INITIAL INVENTORY LOAD ---
    if not os.path.exists(INVENTORY_CSV) or os.path.getsize(INVENTORY_CSV) == 0:
        inv_df = initialize_inventory(prod_df, new_start_date)
        inventory_records_list.extend(inv_df.to_dict('records'))
    else:
        inv_df = pd.read_csv(INVENTORY_CSV)
        inv_df['date'] = pd.to_datetime(inv_df['date']).dt.date
        last_records = (
            inv_df
            .sort_values('date')
            .groupby('product_id')
            .last()
            .reset_index()
        )
        for _, row in last_records.iterrows():
            pid = row['product_id']
            inventory_dict[pid] = {
                'current_stock':       row['end_available_stock'],
                'last_replenishment_date': row['date'],
                'new_stock_arrival':   row['new_stock'],
                'damaged_stock':       row['damaged_stock'],
                'latest_price':        row['price'],
                'purchase_cost':       row['purchasing_cost'],
                'stock_last_change_date': row['date'],
                'sold_stock':          0
            }

    # Seed one day before start so first loop can snapshot previous day
    prev_date = new_start_date - timedelta(days=1)
    # There is no update_inventory_daily function; use update_inventory_record_for_day instead

    inventory_records_list.extend(update_inventory_record_for_day(prev_date, products_dict, inventory_dict))

    # --- MAIN DAILY Loop ---
    cutoff_date = date(2026, 2, 21)
    current_date = new_start_date

    while current_date <= new_end_date:
        # 1) Simulate new customers if past cutoff…
        if current_date >= cutoff_date:
            new_cs, next_customer_id = simulate_new_customers_for_day(
                current_date,
                random.randint(5, 15),
                next_customer_id,
                existing_emails,
                existing_phones
            )
            for cust in new_cs:
                customers_dict[cust['customer_id']] = cust

        # 2) First orders for brand-new registrations…
        eligible = [
            c for c in customers_dict.values()
            if c['registration_date'] <= current_date
        ]
        new_today = [
            c for c in eligible
            if c['registration_date'] == current_date and c.get('total_orders', 0) == 0
        ]
        for cust in new_today:
            order, items, perf, fb, order_id_counter, feedback_id_counter = \
                generate_first_order_for_customer(
                    cust, products, customers_dict,
                    order_id_counter, feedback_id_counter,
                    current_date
                )
            all_orders.append(order)
            all_items.extend(items)
            all_perf.append(perf)
            all_feedback.append(fb)

        # 3) Remaining orders to hit daily target…
        target = get_orders_target_for_date(current_date)
        remaining = max(0, target - sum(1 for o in all_orders if o['order_date'] == current_date))
        if remaining > 0:
            orders, items, perf, fb_list, order_id_counter, feedback_id_counter = \
                generate_orders_for_day(
                    current_date, eligible, products, customers_dict,
                    order_id_counter, feedback_id_counter,
                    remaining
                )
            all_orders.extend(orders)
            all_items.extend(items)
            all_perf.extend(perf)
            all_feedback.extend(fb_list)

        # 4) Update segmentation…
        for cust in eligible:
            customers_dict[cust['customer_id']] = update_segmentation(cust, current_date)

        # 5) END-OF-DAY INVENTORY SNAPSHOT
        daily_records = update_inventory_record_for_day(current_date, products_dict, inventory_dict)
        inventory_records_list.extend(daily_records)

        # — YEAR-END STOCK-LEVEL GROWTH —
        if current_date.month == 12 and current_date.day == 31:
            prod_df = pd.read_csv(STATIC_PRODUCTS_CSV)
            for idx, row in prod_df.iterrows():
                growth = random.uniform(0.20, 0.25)
                prod_df.at[idx, 'max_stock_level'] = int(round(row['max_stock_level'] * (1 + growth)))
                prod_df.at[idx, 'min_stock_level'] = int(round(row['min_stock_level'] * (1 + growth)))
            prod_df.to_csv(STATIC_PRODUCTS_CSV, index=False)

            products = prod_df.to_dict('records')
            products_dict.clear()
            products_dict.update({p['product_id']: p for p in products})
            for pid, inv in inventory_dict.items():
                inv['min_stock_level'] = products_dict[pid]['min_stock_level']
                inv['max_stock_level'] = products_dict[pid]['max_stock_level']
            logging.info(f"Year-end stock levels bumped for {current_date.year}")

        # 6) Move to next day
        current_date += timedelta(days=1)

    # --- FINALIZE & SAVE ---
    updated_customers = [c for c in customers_dict.values()]
    upd_df = pd.DataFrame(updated_customers)
    upd_df['registration_date'] = pd.to_datetime(upd_df['registration_date']).dt.date
    upd_df['last_order_date']   = pd.to_datetime(upd_df['last_order_date']).dt.date
    upd_df.to_csv(STATIC_CUSTOMERS_CSV, index=False)

    inv_new_df = pd.DataFrame(inventory_records_list)
    inv_new_df.to_csv(INVENTORY_CSV, index=False)

    return {
        'orders': pd.DataFrame(all_orders),
        'order_items': pd.DataFrame(all_items),
        'delivery_performance': pd.DataFrame(all_perf),
        'customer_feedback': pd.DataFrame(all_feedback),
    }

# -----------------------
# Append or Save Data
# -----------------------
def append_or_save_data(df, filename):
    try:
        if os.path.exists(filename) and os.path.getsize(filename)>0:
            df.to_csv(filename, mode='a', header=False, index=False)
        else:
            df.to_csv(filename, index=False)
    except Exception as e:
        logging.error(f"Error writing {filename}: {e}")

# -----------------------
# Static Data Generation
# -----------------------
def generate_static_customers(num_customers=5405, start_id=5000):
    customers = []
    start_dt = date(2022, 1, 1)
    end_dt   = date.today()

    # 1) Build a registration-date list:
    reg_dates = []
    reg_dates += [start_dt] * min(200, num_customers)     # first 200 on Jan 1, 2022

    cur_date = start_dt + timedelta(days=1)
    while len(reg_dates) < num_customers and cur_date <= end_dt:
        for _ in range(random.randint(2, 10)):
            if len(reg_dates) < num_customers:
                reg_dates.append(cur_date)
        cur_date += timedelta(days=1)

    # spill over to today if needed
    while len(reg_dates) < num_customers:
        reg_dates.append(end_dt)

    # 2) Generate customer records in chronological order:
    for cid, reg in zip(range(start_id, start_id + num_customers), reg_dates):
        name = fake.name()

        # Email based on name
        local = name.lower().replace(" ", ".")
        domain = random.choice(["gmail.com", "yahoo.com", "outlook.com"])
        email = f"{local}{random.randint(1,999)}@{domain}"

        # Indian mobile number +91 and 10 digits starting 6–9
        phone = f"+91{random.randint(6000000000, 9999999999)}"
        raw = fake.address().replace("\n", ", ")
        # Split into components
        parts = [p.strip() for p in raw.split(",") if p.strip()]
        # street is first part, area is second if exists, else fallback
        street = fake.street_name()
        area = parts[1] if len(parts) > 1 else fake.street_name()
        house_num = fake.building_number()
        city = random.choices(CITY_LIST, weights=CITY_WEIGHTS)[0]
        pincode_low, pincode_high = city_pincode_ranges[city]
        pincode = random.randint(pincode_low, pincode_high)
        full_addr = f"{house_num} {street}, {area}"
        pin = pincode

        customers.append({
            'customer_id': cid,
            'customer_name': name,
            'email': email,
            'phone': phone,
            'address': full_addr,
            'area': area,
            'city': city,
            'pincode': pin,
            'registration_date': reg,
            'total_orders': 0,
            'avg_order_value': 0,
            'last_order_date': reg,
            'segmentation': 'New'
        })

    return pd.DataFrame(customers)


def generate_static_products(num_products_per_category=40):
    fixed_specs = {
        'Fruits & Vegetables':   {'min_stock_level': 2,  'max_stock_level': 18,  'margin_pct_range': (60, 80),  'shelf_life_days': 3},
        'Dairy & Breakfast':     {'min_stock_level': 2,  'max_stock_level': 50,  'margin_pct_range': (15, 25),  'shelf_life_days': 7},
        'Snacks & Munchies':     {'min_stock_level': 20, 'max_stock_level': 600, 'margin_pct_range': (12, 18),  'shelf_life_days': 120},
        'Cold Drinks & Juices':  {'min_stock_level': 10, 'max_stock_level': 250, 'margin_pct_range': (10, 15),  'shelf_life_days': 90},
        'Instant & Frozen Food': {'min_stock_level': 10, 'max_stock_level': 390, 'margin_pct_range': (8, 12),   'shelf_life_days': 90},
        'Grocery & Staples':     {'min_stock_level': 5,  'max_stock_level': 1100,'margin_pct_range': (8, 12),   'shelf_life_days': 90},
        'Household Care':        {'min_stock_level': 10, 'max_stock_level': 660, 'margin_pct_range': (40, 50),  'shelf_life_days': 250},
        'Personal Care':         {'min_stock_level': 25, 'max_stock_level': 500, 'margin_pct_range': (30, 40),  'shelf_life_days': 300},
        'Baby Care':             {'min_stock_level': 10, 'max_stock_level': 160, 'margin_pct_range': (25, 35),  'shelf_life_days': 150},
        'Pet Care':              {'min_stock_level': 8,  'max_stock_level': 140, 'margin_pct_range': (40, 50),  'shelf_life_days': 240},
        'Pharmacy':              {'min_stock_level': 10, 'max_stock_level': 1000,'margin_pct_range': (55, 65),  'shelf_life_days': 300}
    }

    category_products = {
        'Fruits & Vegetables': ['Banana', 'Tomato', 'Onion', 'Potato', 'Mango', 'Apple', 'Cucumber', 'Carrot', 'Spinach', 'Cabbage', 'Ginger', 'Garlic', 'Lemon', 'Coriander', 'Green Chili'],
        'Dairy & Breakfast': ['Amul Milk', 'Mother Dairy Milk', 'Amul Butter', 'Amul Ghee', 'Mother Dairy Paneer', 'Eggs', 'Bread', 'Dahi', 'Mix Dosa', 'Mix Upma', 'Mix Poha', 'Cheese Slices', 'Curd', 'Butter Milk', 'Flavored Yogurt'],
        'Snacks & Munchies': ['Lays Chips', 'Kurkure', 'Haldiram Namkeen', 'Parle-G Biscuits', 'Hide & Seek', 'Bingo Chips', 'Perk', 'Munch', 'Oreo', 'Marie Gold', 'Unibic Cookies', 'Britannia Cake', 'Popcorn', 'Nachos', 'Trail Mix'],
        'Cold Drinks & Juices': ['Coca-Cola', 'Pepsi', 'Thums Up', 'Sprite', 'Fanta', 'Maaza', 'Slice', 'Frooti', 'Real Juice', 'Tropicana', 'Bisleri Water', 'Kinley Water', 'Red Bull', 'Appy Fizz', 'Bournvita'],
        'Instant & Frozen Food': ['Maggi Noodles', 'Yippee Noodles', 'Frozen Paratha', 'Frozen Pizza', 'Frozen French Fries', 'Frozen Momos', 'Ready-to-Eat Meals', 'Instant Pasta', 'Frozen Vegetables', 'Frozen Chicken Nuggets', 'Frozen Fish Fingers', 'Instant Soup', 'Frozen Samosa', 'Frozen Spring Rolls', 'Frozen Paneer Tikka'],
        'Grocery & Staples': ['Aashirvaad Atta', 'Fortune Rice', 'Tata Salt', 'Sugar', 'Toor Dal', 'Chana Dal', 'Moong Dal', 'Masoor Dal', 'Fortune Oil', 'Saffola Oil', 'Tata Tea', 'Nescafe Coffee', 'Catch Spices', 'Everest Masala', 'MDH Masala'],
        'Household Care': ['Surf Excel Detergent', 'Ariel Detergent', 'Vim Dishwash Gel', 'Harpic Toilet Cleaner', 'Lizol Floor Cleaner', 'Colin Glass Cleaner', 'Odonil Air Freshener', 'Domex Disinfectant', 'Dettol Antiseptic', 'Scotch-Brite Scrub Pad', 'Garbage Bags', 'Room Freshener', 'Naphthalene Balls', 'Toilet Brush', 'Mop'],
        'Personal Care': ['Colgate Toothpaste', 'Pepsodent Toothpaste', 'Dabur Red Toothpaste', 'Clinic Plus Shampoo', 'Sunsilk Shampoo', 'Dove Soap', 'Lifebuoy Soap', 'Dettol Soap', 'Nivea Body Lotion', 'Patanjali Face Wash', 'Himalaya Face Wash', 'Veet Hair Removal Cream', 'Whisper Sanitary Pads', 'Stayfree Sanitary Pads', 'Pears Soap'],
        'Baby Care': ['Pampers Diapers', 'Huggies Diapers', 'Johnson Baby Lotion', 'Johnson Baby Shampoo', 'Johnson Baby Powder', 'Baby Wipes', 'Cerelac', 'Farex', 'Lactogen', 'Nestle Nan Pro', 'Baby Oil', 'Baby Cream', 'Baby Soap', 'Baby Feeding Bottle', 'Baby Pacifier'],
        'Pet Care': ['Pedigree Dog Food', 'Whiskas Cat Food', 'Drools Dog Food', 'Me-O Cat Food', 'Pet Shampoo', 'Pet Treats', 'Pet Toys', 'Pet Bed', 'Pet Collar', 'Pet Leash', 'Pet Bowls', 'Pet Litter', 'Pet Cage', 'Pet Vitamins', 'Pet Brush'],
        'Pharmacy': ['Paracetamol', 'Crocin', 'Dolo 650', 'Vicks Vaporub', 'Dabur Honitus', 'Zandu Balm', 'Eno', 'Digene', 'ORS Sachets', 'Band-Aid', 'Dettol Antiseptic Liquid', 'Thermometer', 'Sanitizer', 'Face Mask', 'Multivitamin Tablets']
    }

    product_mrp_map = {'Banana':32, 'Tomato':28, 'Onion':40, 'Potato':20, 'Mango':30, 'Apple':100, 'Cucumber':26, 'Carrot':25, 'Spinach':20, 'Cabbage':20, 'Ginger':120, 'Garlic':160, 'Lemon':100, 'Coriander':80, 'Green Chili':90, 'Amul Milk':40, 'Mother Dairy Milk':52, 'Amul Butter':56, 'Amul Ghee':520, 'Mother Dairy Paneer':70, 'Eggs':84, 'Bread':35, 'Dahi':50, 'Mix Dosa':30, 'Mix Upma':30, 'Mix Poha':30, 'Cheese Slices':80, 'Curd':50, 'Butter Milk':35, 'Flavored Yogurt':25, 'Lays Chips':30, 'Kurkure':20, 'Haldiram Namkeen':50, 'Parle-G Biscuits':40, 'Hide & Seek':20, 'Bingo Chips':20, 'Perk':20, 'Munch':20, 'Oreo':45, 'Marie Gold':20, 'Unibic Cookies':30, 'Britannia Cake':20, 'Popcorn':25, 'Nachos':25, 'Trail Mix':120, 'Coca-Cola':40, 'Pepsi':40, 'Thums Up':35, 'Sprite':40, 'Fanta':40, 'Maaza':30, 'Slice':30, 'Frooti':25, 'Real Juice':150, 'Tropicana':170, 'Bisleri Water':20, 'Kinley Water':20, 'Red Bull':80, 'Appy Fizz':40, 'Bournvita':200, 'Maggi Noodles':20, 'Yippee Noodles':20, 'Frozen Paratha':80, 'Frozen Pizza':150, 'Frozen French Fries':100, 'Frozen Momos':120, 'Ready-to-Eat Meals':180, 'Instant Pasta':60, 'Frozen Vegetables':80, 'Frozen Chicken Nuggets':160, 'Frozen Fish Fingers':140, 'Instant Soup':50, 'Frozen Samosa':40, 'Frozen Spring Rolls':50, 'Frozen Paneer Tikka':200, 'Aashirvaad Atta':55, 'Fortune Rice':60, 'Tata Salt':30, 'Sugar':45, 'Toor Dal':150, 'Chana Dal':120, 'Moong Dal':140, 'Masoor Dal':130, 'Fortune Oil':180, 'Saffola Oil':200, 'Tata Tea':80, 'Nescafe Coffee':100, 'Catch Spices':60, 'Everest Masala':50, 'MDH Masala':45, 'Surf Excel Detergent':60, 'Ariel Detergent':90, 'Vim Dishwash Gel':80, 'Harpic Toilet Cleaner':70, 'Lizol Floor Cleaner':80, 'Colin Glass Cleaner':70, 'Odonil Air Freshener':40, 'Domex Disinfectant':80, 'Dettol Antiseptic':120, 'Scotch-Brite Scrub Pad':40, 'Garbage Bags':60, 'Room Freshener':80, 'Naphthalene Balls':30, 'Toilet Brush':150, 'Mop':200, 'Colgate Toothpaste':80, 'Pepsodent Toothpaste':60, 'Dabur Red Toothpaste':60, 'Clinic Plus Shampoo':80, 'Sunsilk Shampoo':90, 'Dove Soap':70, 'Lifebuoy Soap':25, 'Dettol Soap':40, 'Nivea Body Lotion':200, 'Patanjali Face Wash':60, 'Himalaya Face Wash':80, 'Veet Hair Removal Cream':150, 'Whisper Sanitary Pads':60, 'Stayfree Sanitary Pads':50, 'Pears Soap':50, 'Pampers Diapers':300, 'Huggies Diapers':250, 'Johnson Baby Lotion':150, 'Johnson Baby Shampoo':120, 'Johnson Baby Powder':100, 'Baby Wipes':80, 'Cerelac':150, 'Farex':200, 'Lactogen':150, 'Nestle Nan Pro':500, 'Baby Oil':100, 'Baby Cream':200, 'Baby Soap':30, 'Baby Feeding Bottle':150, 'Baby Pacifier':100, 'Pedigree Dog Food':200, 'Whiskas Cat Food':220, 'Drools Dog Food':180, 'Me-O Cat Food':200, 'Pet Shampoo':150, 'Pet Treats':50, 'Pet Toys':200, 'Pet Bed':500, 'Pet Collar':100, 'Pet Leash':150, 'Pet Bowls':200, 'Pet Litter':250, 'Pet Cage':1500, 'Pet Vitamins':300, 'Pet Brush':100, 'Paracetamol':20, 'Crocin':30, 'Dolo 650':35, 'Vicks Vaporub':50, 'Dabur Honitus':40, 'Zandu Balm':30, 'Eno':20, 'Digene':20, 'ORS Sachets':20, 'Band-Aid':40, 'Dettol Antiseptic Liquid':120, 'Thermometer':200, 'Sanitizer':100, 'Face Mask':50, 'Multivitamin Tablets':200}


    products = []
    product_id_counter = 1000

    for cat, items in category_products.items():
        specs = fixed_specs[cat]
        for name in items:
            mrp = product_mrp_map[name]
            low_margin, high_margin = specs['margin_pct_range']
            base_margin_pct = round(random.uniform(low_margin, high_margin), 2)
            base_purchase_cost = round(mrp * (1 - base_margin_pct/100), 2)
            products.append({
                'product_id': product_id_counter,
                'product_name': name,
                'category': cat,
                'brand': fake.company(),
                'mrp': mrp,
                'base_margin_pct': base_margin_pct,
                'base_purchasing_cost': base_purchase_cost,
                'shelf_life_days': specs['shelf_life_days'],
                'min_stock_level': specs['min_stock_level'],
                'max_stock_level': specs['max_stock_level']
            })
            product_id_counter += 1

    return pd.DataFrame(products)

def generate_marketing_performance(start_date_str='2022-01-01', end_date=None):
    if end_date is None:
        end_date = date.today()
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d').date()
    date_range_days = (end_date - start_date).days
    segmentation_list = ["Premium", "Inactive", "New", "Regular"]
    campaign_info = {
        "Premium": {"campaign_name": "Exclusive Premium Offers", "frequency": 1, "channel": "Email"},
        "Inactive": {"campaign_name": "We Miss You - Come Back", "frequency": 3, "channel": "SMS"},
        "New": {"campaign_name": "Welcome Offer", "frequency": 2, "channel": "App"},
        "Regular": {"campaign_name": "Thank You - Special Deals", "frequency": 2, "channel": "Email"}
    }
    marketing_data = []
    campaign_id = 1
    for day in range(0, date_range_days + 1, 7):
        current_date = start_date + timedelta(days=day)
        for seg in segmentation_list:
            info = campaign_info[seg]
            spend = round(random.uniform(40000, 70000), 2)
            revenue = round(random.uniform(80000, 100000), 2)
            marketing_data.append({
                'campaign_id': campaign_id,
                'campaign_name': info['campaign_name'],
                'date': current_date - timedelta(days=1),
                'target_segmentation': seg,
                'frequency': info['frequency'],
                'channel': info['channel'],
                'impressions': random.randint(2000, 4000),
                'clicks': random.randint(900, 2000),
                'conversions': random.randint(100, 300),
                'spend': spend,
                'revenue_generated': revenue,
                'roas': round(revenue / spend, 2)
            })
            campaign_id += 1
    return pd.DataFrame(marketing_data)


# -----------------------
# Main Execution
# -----------------------
if __name__ == '__main__':
    # Generate static customers
    if not os.path.exists(STATIC_CUSTOMERS_CSV) or os.path.getsize(STATIC_CUSTOMERS_CSV) == 0:
        cust_df = generate_static_customers()
        cust_df.to_csv(STATIC_CUSTOMERS_CSV, index=False)

    # Generate static products
    if not os.path.exists(STATIC_PRODUCTS_CSV) or os.path.getsize(STATIC_PRODUCTS_CSV) == 0:
        prod_df = generate_static_products()
        prod_df.to_csv(STATIC_PRODUCTS_CSV, index=False)

    # Generate marketing performance
    if not os.path.exists(MARKETING_PERFORMANCE_CSV) or os.path.getsize(MARKETING_PERFORMANCE_CSV) == 0:
        mp_df = generate_marketing_performance(DEFAULT_START_DATE, date.today())
        mp_df.to_csv(MARKETING_PERFORMANCE_CSV, index=False)

    # Initialize inventory if needed
    if not os.path.exists(INVENTORY_CSV) or os.path.getsize(INVENTORY_CSV) == 0:
        static_prod_df = pd.read_csv(STATIC_PRODUCTS_CSV)
        inv_df = initialize_inventory(static_prod_df, datetime.strptime(DEFAULT_START_DATE, '%Y-%m-%d').date())
        inv_df.to_csv(INVENTORY_CSV, index=False)

    # Generate and append daily orders
    start_date, end_date = get_new_date_range(ORDERS_CSV, DEFAULT_START_DATE)
    if start_date <= end_date:
        # Unchanged: generate orders, order_items, delivery_performance, customer_feedback
        new_data = generate_new_daily_orders(start_date, end_date)
        # Append orders etc.
        append_or_save_data(new_data['orders'], ORDERS_CSV)
        append_or_save_data(new_data['order_items'], ORDER_ITEMS_CSV)
        append_or_save_data(new_data['delivery_performance'], DELIVERY_PERFORMANCE_CSV)
        append_or_save_data(new_data['customer_feedback'], CUSTOMER_FEEDBACK_CSV)
        # After orders for each day, update inventory:
        current = start_date
        all_inv_records = []
        while current <= end_date:
            daily_rows = update_inventory_record_for_day(current, products_dict, inventory_dict)
            all_inv_records.extend(daily_rows)
            current += timedelta(days=1)
        inv_new_df = pd.DataFrame(all_inv_records)
        inv_new_df = inv_new_df[[
            'product_id', 'date', 'mrp', 'price', 'purchasing_cost', 'Selling Cost',
            'profit', 'new_stock', 'start_available_stock', 'sold_stock',
            'end_available_stock', 'damaged_stock'
        ]]
        inv_new_df.to_csv(INVENTORY_CSV, mode='a', header=False, index=False)
    else:
        logging.info("No new days to process for orders.")
    # Clean up seed inventory row for 2021-12-31 after file generation
if os.path.exists(INVENTORY_CSV):
    inventory_df = pd.read_csv(INVENTORY_CSV, parse_dates=['date'])
    inventory_df = inventory_df[inventory_df['date'] != pd.Timestamp("2021-12-31")]
    inventory_df.to_csv(INVENTORY_CSV, index=False)

2025-06-19 09:50:18,333 INFO:Year-end stock levels bumped for 2022
2025-06-19 09:51:46,889 INFO:Year-end stock levels bumped for 2023
2025-06-19 09:53:44,309 INFO:Year-end stock levels bumped for 2024


##### Check/Test

In [13]:
import pandas as pd

# 1) Load your data
inv_df  = pd.read_csv('Apna_inventory.csv', parse_dates=['date'])
prod_df = pd.read_csv('Apna_products.csv')

# 2) Attach category to each inventory row
inv_cat = inv_df.merge(
    prod_df[['product_id', 'category']],
    on='product_id',
    how='left'
)

# 3) Aggregate per category
#    - total_received_stock: sum of all 'new_stock' arrivals (includes initial stock)
#    - total_damaged_stock : sum of all 'damaged_stock' recorded
received = (
    inv_cat
    .groupby('category', as_index=False)['new_stock']
    .sum()
    .rename(columns={'new_stock': 'total_received_stock'})
)
damaged = (
    inv_cat
    .groupby('category', as_index=False)['damaged_stock']
    .sum()
    .rename(columns={'damaged_stock': 'total_damaged_stock'})
)

# 4) Merge and compute damage percentage
agg = pd.merge(received, damaged, on='category')
agg['damage_percent'] = (
    agg['total_damaged_stock'] / agg['total_received_stock']
) * 100

# 5) Which categories hit your 8–10% damage target?
within_8_10 = agg[
    (agg['damage_percent'] >= 8.0) &
    (agg['damage_percent'] <= 10.0)
]

# 6) Display results
print("\nCategory-level damage percentages:")
print(agg[['category','total_received_stock','total_damaged_stock','damage_percent']]
      .to_string(index=False))

print("\nCategories with damage between 8% and 10%:")
print(within_8_10[['category','damage_percent']].to_string(index=False))



Category-level damage percentages:
             category  total_received_stock  total_damaged_stock  damage_percent
            Baby Care                 26528                 2132        8.036791
 Cold Drinks & Juices                 73406                 1609        2.191919
    Dairy & Breakfast                205553                 4986        2.425652
  Fruits & Vegetables                168504                12119        7.192114
    Grocery & Staples                309884                 9469        3.055660
       Household Care                 67828                 3288        4.847556
Instant & Frozen Food                111850                 3572        3.193563
        Personal Care                 42500                 3033        7.136471
             Pet Care                 14387                 1540       10.704108
             Pharmacy                 83362                 4873        5.845589
    Snacks & Munchies                130078                 4731        3

In [14]:
# master table

In [15]:
import os
import pandas as pd

def main():
    # -----------------------
    # File Paths
    # -----------------------
    ORDERS_CSV = 'Apna_orders.csv'
    ORDER_ITEMS_CSV = 'Apna_order_items.csv'
    DELIVERY_PERFORMANCE_CSV = 'Apna_delivery_performance.csv'
    CUSTOMER_FEEDBACK_CSV = 'Apna_customer_feedback.csv'
    STATIC_PRODUCTS_CSV = 'Apna_products.csv'
    STATIC_CUSTOMERS_CSV = 'Apna_customers.csv'
    
    # External lookup files (Excel)
    RATING_ICONS_XLSX = 'rating_Icon.xlsx'       # Contains: Rating, Emoji, Star
    CATEGORY_ICONS_XLSX = 'category_Icons.xlsx'   # Contains: category, Img
    
    # Output Master Table CSV file
    MASTER_TABLE_CSV = 'Master_Table.csv'
    
    # -----------------------
    # Read Input Files with proper parsing
    # -----------------------
    orders = pd.read_csv(ORDERS_CSV, parse_dates=['order_date'])
    order_items = pd.read_csv(ORDER_ITEMS_CSV)
    delivery_perf = pd.read_csv(DELIVERY_PERFORMANCE_CSV, parse_dates=['promised_time', 'actual_time'])
    customer_fb = pd.read_csv(CUSTOMER_FEEDBACK_CSV, parse_dates=['feedback_date'])
    products = pd.read_csv(STATIC_PRODUCTS_CSV)
    customers = pd.read_csv(STATIC_CUSTOMERS_CSV, parse_dates=['registration_date', 'last_order_date'])
    
    # Read Excel lookup tables
    rating_icons = pd.read_excel(RATING_ICONS_XLSX, engine='openpyxl')
    category_icons = pd.read_excel(CATEGORY_ICONS_XLSX, engine='openpyxl')
    
    # -----------------------
    # Load Existing Master Table (if available)
    # -----------------------
    if os.path.exists(MASTER_TABLE_CSV) and os.path.getsize(MASTER_TABLE_CSV) > 0:
        master_df_existing = pd.read_csv(MASTER_TABLE_CSV, parse_dates=['order_date'])
        last_master_date = master_df_existing['order_date'].max()
        print("Existing master table found. Last order date:", last_master_date)
    else:
        master_df_existing = None
        last_master_date = None
        print("No existing master table found. Creating a new one.")
    
    # -----------------------
    # Filter for New Orders
    # -----------------------
    if last_master_date is not None:
        new_orders = orders[orders['order_date'] > last_master_date].copy()
    else:
        new_orders = orders.copy()
    
    if new_orders.empty:
        print("No new orders found to append.")
        return
    
    # -----------------------
    # Merge DataFrames to Build New Data Portion
    # -----------------------
    # 1. Merge new orders with order_items on order_id.
    df_new = new_orders.merge(order_items, on='order_id', how='left')
    
    # 2. Merge with products on product_id.
    df_new = df_new.merge(products, on='product_id', how='left', suffixes=('', '_prod'))
    
    # 3. Rename customers' 'segmentation' column and merge on customer_id.
    customers.rename(columns={'segmentation': 'customer_segment'}, inplace=True)
    df_new = df_new.merge(customers, on='customer_id', how='left', suffixes=('', '_cust'))
    
    # 4. Merge with customer feedback on order_id and customer_id.
    df_new = df_new.merge(customer_fb, on=['order_id', 'customer_id'], how='left', suffixes=('', '_fb'))
    
    # 5. Merge with delivery performance on order_id.
    df_new = df_new.merge(delivery_perf, on='order_id', how='left', suffixes=('', '_dp'))
    
    # 6. Merge with rating icons (to add Emoji and Star) using rating.
    df_new = df_new.merge(rating_icons, left_on='rating', right_on='Rating', how='left')
    
    # 7. Merge with category icons on product category.
    df_new = df_new.merge(category_icons, on='category', how='left')
    
    # -----------------------
    # Prepare New Master Data
    # -----------------------
    # Create a date-only column (from order_date)
    df_new['Date'] = df_new['order_date'].dt.date

    # Select and rename columns.
    # Note: The original selection expected a 'price' column but order_items provides 'unit_price'.
    # We select 'unit_price' and then rename it to 'price' for consistency.
    master_new = df_new[['Date',
                         'order_id',
                         'order_date',
                         'order_status',
                         'delivery_partner_id',
                         'payment_method',
                         'customer_name',
                         'city',
                         'customer_segment',
                         'pincode',
                         'customer_id',
                         'product_id',
                         'product_name',
                         'category',
                         'unit_price',
                         'feedback_id',
                         'quantity',
                         'total_price',   # Will be renamed to 'Value'
                         'rating',
                         'feedback_category',
                         'sentiment',
                         'feedback_text',
                         'delivery_status',
                         'promised_time',
                         'actual_time',
                         'delivery_time_minutes',
                         'reasons_if_delayed',
                         'Img',
                         'Emoji',
                         'Star'               
                         ]].copy()
    
    # Rename columns as needed
    master_new.rename(columns={'unit_price': 'price', 'total_price': 'Value'}, inplace=True)
    
    # -----------------------
    # Enforce Correct Data Types
    # -----------------------
    master_new['order_date'] = pd.to_datetime(master_new['order_date'], errors='coerce')
    if 'promised_time' in master_new.columns:
        master_new['promised_time'] = pd.to_datetime(master_new['promised_time'], errors='coerce')
    if 'actual_time' in master_new.columns:
        master_new['actual_time'] = pd.to_datetime(master_new['actual_time'], errors='coerce')
    
    for col in ['price', 'quantity', 'Value', 'delivery_time_minutes']:
        if col in master_new.columns:
            master_new[col] = pd.to_numeric(master_new[col], errors='coerce')
    
    # -----------------------
    # Append New Data to Existing Master Table
    # -----------------------
    if master_df_existing is not None:
        master_df_existing['order_date'] = pd.to_datetime(master_df_existing['order_date'], errors='coerce')
        master_updated = pd.concat([master_df_existing, master_new], ignore_index=True)
    else:
        master_updated = master_new

    master_updated.sort_values(by='order_date', inplace=True)
    
    print("Data types in the master table:")
    print(master_updated.dtypes)
    
    master_updated.to_csv(MASTER_TABLE_CSV, index=False)
    print("Master table updated and saved to", MASTER_TABLE_CSV)

if __name__ == "__main__":
    main()


No existing master table found. Creating a new one.
Data types in the master table:
Date                             object
order_id                          int64
order_date               datetime64[ns]
order_status                     object
delivery_partner_id               int64
payment_method                   object
customer_name                    object
city                             object
customer_segment                 object
pincode                           int64
customer_id                       int64
product_id                        int64
product_name                     object
category                         object
price                             int64
feedback_id                       int64
quantity                          int64
Value                             int64
rating                            int64
feedback_category                object
sentiment                        object
feedback_text                    object
delivery_status                  obj

In [16]:
# inventory master

In [17]:
import pandas as pd
import os

# -----------------------
# File Paths
# -----------------------
STATIC_PRODUCTS_CSV = 'Apna_products.csv'
INVENTORY_CSV = 'Apna_inventory.csv'
INVENTORY_MASTER_CSV = 'Inventory_Master_Table.csv'  # Using CSV for the master inventory

# -----------------------
# Read the Data Files
# -----------------------
# Read the products data
products = pd.read_csv(STATIC_PRODUCTS_CSV)

# Read the inventory data, parsing the 'date' column as a datetime.
inventory = pd.read_csv(INVENTORY_CSV, parse_dates=['date'])

# -----------------------
# Load Existing Master Inventory (if available)
# -----------------------
if os.path.exists(INVENTORY_MASTER_CSV) and os.path.getsize(INVENTORY_MASTER_CSV) > 0:
    master_inventory_existing = pd.read_csv(INVENTORY_MASTER_CSV, parse_dates=['date'])
    last_inventory_date = master_inventory_existing['date'].max()
    print("Existing master inventory found. Last inventory date:", last_inventory_date)
else:
    master_inventory_existing = None
    last_inventory_date = None
    print("No existing master inventory found. Creating a new one.")

# -----------------------
# Filter for New Inventory Data
# -----------------------
if last_inventory_date is not None:
    new_inventory = inventory[inventory['date'] > last_inventory_date].copy()
else:
    new_inventory = inventory.copy()

if new_inventory.empty:
    print("No new inventory data to append. Exiting.")
    exit(0)

# -----------------------
# Merge New Inventory Data with Products
# -----------------------
# Join new inventory records with product details based on product_id.
inventory_master_new = pd.merge(
    new_inventory,
    products,
    on='product_id',
    how='left',
    suffixes=('', '_prod')
)

# -----------------------
# Select and Order Columns for the Master Inventory Table
# -----------------------
selected_columns = [
    'date',                  
    'product_id',            
    'product_name',          
    'category',              
    'price',                 
    'mrp',                   
    'purchasing_cost',       
    'Selling Cost',
    'profit',
    'new_stock',
    'start_available_stock',
    'sold_stock',
    'end_available_stock',
    'damaged_stock'
]

inventory_master_new = inventory_master_new[selected_columns]

# -----------------------
# Append New Data to the Existing Master Inventory Table
# -----------------------
if master_inventory_existing is not None:
    master_inventory_updated = pd.concat(
        [master_inventory_existing, inventory_master_new],
        ignore_index=True
    )
else:
    master_inventory_updated = inventory_master_new

# -----------------------
# ROUND & CAST ALL NUMERIC COLUMNS TO INTEGER
# -----------------------
# Identify all numeric columns
num_cols = master_inventory_updated.select_dtypes(include='number').columns

# Round to nearest integer and cast to int
master_inventory_updated[num_cols] = (
    master_inventory_updated[num_cols]
    .round(0)
    .astype(int)
)

# -----------------------
# Save the Updated Master Inventory Table to CSV
# -----------------------
master_inventory_updated.to_csv(INVENTORY_MASTER_CSV, index=False)
print("Inventory master table has been updated and saved to", INVENTORY_MASTER_CSV)


No existing master inventory found. Creating a new one.
Inventory master table has been updated and saved to Inventory_Master_Table.csv
